In [0]:
pathdf1 = '/FileStore/tables/country_vaccinations.csv'

In [0]:
# Opções de leitura de arquivo

#df1 = spark.read.format('csv').option('header','true').load(pathdf1)
#df1 = spark.read.csv(pathdf1)
df1 = spark.read.option('header','true').option('delimiter', ',').option('inferSchema','true').csv(pathdf1) #sep no lugar de delimiter também funciona

# option('inferSchemas','true') para que ele tente definir os tipos de dados das colunas

'''
Outro formatos
spark.read.json...
spark.read.orc
spark.read.parquet

para json estruturado existe a opção de option('multiline','true')
'''

Out[2]: "\nOutro formatos\nspark.read.json...\nspark.read.orc\nspark.read.parquet\n\npara json estruturado existe a opção de option('multiline','true')\n"

In [0]:
# Exibindo df

df1.show(10) #printa o log
#df1.take(1) #transforma numa lista (ou matriz) python
#display(df1) não é do spark, função da databricks para exibição

+-----------+--------+----------+------------------+-----------------+-----------------------+----------------------+------------------+------------------------------+-----------------------------+-----------------------------------+------------------------------+--------------------+--------------------+--------------------+
|    country|iso_code|      date|total_vaccinations|people_vaccinated|people_fully_vaccinated|daily_vaccinations_raw|daily_vaccinations|total_vaccinations_per_hundred|people_vaccinated_per_hundred|people_fully_vaccinated_per_hundred|daily_vaccinations_per_million|            vaccines|         source_name|      source_website|
+-----------+--------+----------+------------------+-----------------+-----------------------+----------------------+------------------+------------------------------+-----------------------------+-----------------------------------+------------------------------+--------------------+--------------------+--------------------+
|Afghanistan|   

In [0]:
#df1.dtypes #inferSchema
df1.printSchema

Out[4]: <bound method DataFrame.printSchema of DataFrame[country: string, iso_code: string, date: string, total_vaccinations: double, people_vaccinated: double, people_fully_vaccinated: double, daily_vaccinations_raw: double, daily_vaccinations: double, total_vaccinations_per_hundred: double, people_vaccinated_per_hundred: double, people_fully_vaccinated_per_hundred: double, daily_vaccinations_per_million: double, vaccines: string, source_name: string, source_website: string]>

In [0]:
#df1.write.format('parquet').save('/FileStore/tables/raw_zone_parquet/')
df1.repartition(2).write.format('parquet').mode('overwrite').save('/FileStore/tables/raw_zone_parquet/')

In [0]:
#rdd = sc.textFile(pathrdd)
#rdd.collect() #rdd.show() -> Errado, .show() sómente para df

In [0]:
nome_tabela_temporaria = 'tempTableDataFrame1'
df1.createOrReplaceTempView(nome_tabela_temporaria)

In [0]:
spark.read.table(nome_tabela_temporaria).show()

+-----------+--------+----------+------------------+-----------------+-----------------------+----------------------+------------------+------------------------------+-----------------------------+-----------------------------------+------------------------------+--------------------+--------------------+--------------------+
|    country|iso_code|      date|total_vaccinations|people_vaccinated|people_fully_vaccinated|daily_vaccinations_raw|daily_vaccinations|total_vaccinations_per_hundred|people_vaccinated_per_hundred|people_fully_vaccinated_per_hundred|daily_vaccinations_per_million|            vaccines|         source_name|      source_website|
+-----------+--------+----------+------------------+-----------------+-----------------------+----------------------+------------------+------------------------------+-----------------------------+-----------------------------------+------------------------------+--------------------+--------------------+--------------------+
|Afghanistan|   

In [0]:
#spark.sql('SELECT * FROM tempTableDataFrame1').show()
spark.sql('SELECT count(*) tt, country FROM tempTableDataFrame1 GROUP BY country').show()

+---+--------------------+
| tt|             country|
+---+--------------------+
|151|                Chad|
|282|            Anguilla|
|201|            Djibouti|
|237|                Fiji|
|321|             Germany|
|197|             Comoros|
|256|         Afghanistan|
|277|            Cambodia|
|320|              France|
|320|              Greece|
|169|British Virgin Is...|
|275|            Dominica|
|284|             Algeria|
|240|   Equatorial Guinea|
|319|           Argentina|
|252|              Angola|
|319|             Belgium|
|289|             Ecuador|
|146|Bonaire Sint Eust...|
|306|             Albania|
+---+--------------------+
only showing top 20 rows



In [0]:
# Scala
#import org.apache.spark.sql.functions._

from pyspark.sql.functions import col, column, lit, expr, upper, lower

df1.select(col('country'), col('date'), column('iso_code')).show() #nenhuma diferença entre col e column

+-----------+----------+--------+
|    country|      date|iso_code|
+-----------+----------+--------+
|Afghanistan|2021-02-22|     AFG|
|Afghanistan|2021-02-23|     AFG|
|Afghanistan|2021-02-24|     AFG|
|Afghanistan|2021-02-25|     AFG|
|Afghanistan|2021-02-26|     AFG|
|Afghanistan|2021-02-27|     AFG|
|Afghanistan|2021-02-28|     AFG|
|Afghanistan|2021-03-01|     AFG|
|Afghanistan|2021-03-02|     AFG|
|Afghanistan|2021-03-03|     AFG|
|Afghanistan|2021-03-04|     AFG|
|Afghanistan|2021-03-05|     AFG|
|Afghanistan|2021-03-06|     AFG|
|Afghanistan|2021-03-07|     AFG|
|Afghanistan|2021-03-08|     AFG|
|Afghanistan|2021-03-09|     AFG|
|Afghanistan|2021-03-10|     AFG|
|Afghanistan|2021-03-11|     AFG|
|Afghanistan|2021-03-12|     AFG|
|Afghanistan|2021-03-13|     AFG|
+-----------+----------+--------+
only showing top 20 rows



In [0]:
'''
Definindo Schema manualmente

from pyspark.sql.types import *

dataframe = StructType([
    StructField('colStr1', StringType()),
    StructField('colInt2', IntegerType()),
    SturctField('colDecimal3', DecimalType())
])
'''

Out[11]: "\nDefinindo Schema manualmente\n\nfrom pyspark.sql.types import *\n\ndataframe = StructType([\n    StructField('colStr1', StringType()),\n    StructField('colInt2', IntegerType()),\n    SturctField('colDecimal3', DecimalType())\n])\n"

In [0]:
'''
Define uma nova coluna com um valor constante
df2 = df1.withColumn('nova_coluna', lit(1)) # lit é do pyspark.sql.functions
df2.select('country', df2.nova_coluna).show(5)

# Adiciona coluna
teste = expr('total_vaccinations < 40') # também do functions
df1.select('country','total_vaccinations').withColumn('test', teste).show(5)
'''
# Renomear uma coluna
df1.select(expr('total_vaccinations as total_de_vacinados')).show(5)
'''
df1.select(col('country').alias('pais')).show(5)

df1.select('country').withColumnRenamed('country','pais').show(5)

# Remover colunas
df3 = df1.drop('country')
df3.columns
'''

+------------------+
|total_de_vacinados|
+------------------+
|               0.0|
|              null|
|              null|
|              null|
|              null|
+------------------+
only showing top 5 rows

Out[12]: "\ndf1.select(col('country').alias('pais')).show(5)\n\ndf1.select('country').withColumnRenamed('country','pais').show(5)\n\n# Remover colunas\ndf3 = df1.drop('country')\ndf3.columns\n"

In [0]:
# Filtrando e ordenando dados
# where() e filter() fazem a mesma coisa

#display(df1.filter(df1.total_vaccinations > 55).orderBy(df1.total_vaccinations))
#display(df1.select(df1.country, df1.date, df1.total_vaccinations).where(df1.country.isin('Brazil', 'Argentina')))
df1.select(df1.country).distinct().show(10)

+-------------------+
|            country|
+-------------------+
|           Anguilla|
|        Afghanistan|
|            Algeria|
|          Argentina|
|             Angola|
|            Albania|
|            Andorra|
|              Aruba|
|            Armenia|
|Antigua and Barbuda|
+-------------------+
only showing top 10 rows



In [0]:
# Convertendo dados
print(df1.dtypes[3], '\n')
df4 = df1.withColumn('total_vaccinations_str', df1.total_vaccinations.cast('string'))
df4.dtypes

('total_vaccinations', 'double') 

Out[14]: [('country', 'string'),
 ('iso_code', 'string'),
 ('date', 'string'),
 ('total_vaccinations', 'double'),
 ('people_vaccinated', 'double'),
 ('people_fully_vaccinated', 'double'),
 ('daily_vaccinations_raw', 'double'),
 ('daily_vaccinations', 'double'),
 ('total_vaccinations_per_hundred', 'double'),
 ('people_vaccinated_per_hundred', 'double'),
 ('people_fully_vaccinated_per_hundred', 'double'),
 ('daily_vaccinations_per_million', 'double'),
 ('vaccines', 'string'),
 ('source_name', 'string'),
 ('source_website', 'string'),
 ('total_vaccinations_str', 'string')]

In [0]:
# funções upper() e lower()

df1.select(df1.country).distinct().show()
df1.select(upper(df1.country)).distinct().show()
df1.select(lower(df1.country)).distinct().show()

+-------------------+
|            country|
+-------------------+
|           Anguilla|
|        Afghanistan|
|            Algeria|
|          Argentina|
|             Angola|
|            Belgium|
|            Albania|
|              Benin|
|            Bahamas|
|            Belarus|
|            Andorra|
|         Bangladesh|
|           Barbados|
|              Aruba|
|         Azerbaijan|
|            Armenia|
|             Belize|
|Antigua and Barbuda|
|          Australia|
|            Austria|
+-------------------+
only showing top 20 rows

+-------------------+
|     upper(country)|
+-------------------+
|            ARMENIA|
|            BAHAMAS|
|         BANGLADESH|
|            ALBANIA|
|            BELARUS|
|ANTIGUA AND BARBUDA|
|         AZERBAIJAN|
|            ANDORRA|
|            ALGERIA|
|              ARUBA|
|              BENIN|
|           ANGUILLA|
|            AUSTRIA|
|             BELIZE|
|          ARGENTINA|
|             ANGOLA|
|            BELGIUM|
|     

In [0]:
# Dataframe genérico

d = [{'name': 'Alana', 'age': 4}]
df_g = spark.createDataFrame(d)
df_g.show()

+---+-----+
|age| name|
+---+-----+
|  4|Alana|
+---+-----+



In [0]:
# Joins

rdd1 = [
    {'name': 'Henrique', 'age': 31, 'status': 'true'},
    {'name': 'do Val', 'age': 33, 'status': 'true'},
    {'name': 'Silva', 'age': 34, 'status': 'false'},
    {'name': 'Rafaela', 'age': 32, 'status': 'true'},
    {'name': 'Scarlat', 'age': 35, 'status': 'false'},
    {'name': 'Bonuzzi', 'age': 36, 'status': 'true'},
]
dff1 = spark.createDataFrame(rdd1)
dff1.show()

rdd2 = [
    {'name': 'do Val', 'country': 'Portugal'},
    {'name': 'Americo', 'country': 'Spain'},
    {'name': 'Bonuzzi', 'country': 'Italia'},
    {'name': 'Henrique', 'country': 'Brazil'},
    {'name': 'Silva', 'country': 'Brazil'},
]
dff2 = spark.createDataFrame(rdd2)
dff2.show()

join_type = 'full_outer'
join_condition = dff1.name == dff2.name
df5 = dff1.join(dff2, join_condition, join_type)
df5.show()

'''join types
left_semi: 
right_outer:
left_outer:
full_outer:
left_anti:
cross'''

+---+--------+------+
|age|    name|status|
+---+--------+------+
| 31|Henrique|  true|
| 33|  do Val|  true|
| 34|   Silva| false|
| 32| Rafaela|  true|
| 35| Scarlat| false|
| 36| Bonuzzi|  true|
+---+--------+------+

+--------+--------+
| country|    name|
+--------+--------+
|Portugal|  do Val|
|   Spain| Americo|
|  Italia| Bonuzzi|
|  Brazil|Henrique|
|  Brazil|   Silva|
+--------+--------+

+----+--------+------+--------+--------+
| age|    name|status| country|    name|
+----+--------+------+--------+--------+
|null|    null|  null|   Spain| Americo|
|  36| Bonuzzi|  true|  Italia| Bonuzzi|
|  31|Henrique|  true|  Brazil|Henrique|
|  32| Rafaela|  true|    null|    null|
|  35| Scarlat| false|    null|    null|
|  34|   Silva| false|  Brazil|   Silva|
|  33|  do Val|  true|Portugal|  do Val|
+----+--------+------+--------+--------+

Out[17]: 'join types\nleft_semi: \nright_outer:\nleft_outer:\nfull_outer:\nleft_anti:\ncross'